In [1]:
import numpy as np
#standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit.providers.aer import QasmSimulator
from qiskit_nature.drivers.second_quantization import PySCFDriver
from qiskit_nature.transformers.second_quantization.electronic import FreezeCoreTransformer

# This is a script I wrote to better manipulate the measurement process
from Measure import SingleMultip, MultiMultip, MeasureCircuit, evaluation, evaluation_IZ

In [3]:
molecule = 'Li 0.0 0.0 0.0; H 0.0 0.0 1.5'
driver = PySCFDriver(atom=molecule)

In [4]:
qmolecule = driver.run()

In [5]:
fct = [FreezeCoreTransformer(freeze_core=True, remove_orbitals=[3,4])]

In [6]:
from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem
problem = ElectronicStructureProblem(driver, fct)
second_q_ops = problem.second_q_ops()

# Hamiltonian
main_op = second_q_ops[0]
from qiskit_nature.mappers.second_quantization import ParityMapper
from qiskit_nature.converters.second_quantization.qubit_converter import QubitConverter

converter = QubitConverter(mapper=ParityMapper(), two_qubit_reduction=True)

# Mapping Fermions to Qubits
#num_particles = (problem.molecule_data_transformed.num_alpha,
             #problem.molecule_data_transformed.num_beta)
num_particles = problem.num_particles
qubit_op = converter.convert(main_op, num_particles = problem.num_particles)
from qiskit_nature.circuit.library import HartreeFock


num_spin_orbitals = problem.num_spin_orbitals
init_state = HartreeFock(num_spin_orbitals, num_particles, converter)

In [8]:
from qiskit.circuit.library import TwoLocal
from qiskit_nature.circuit.library import UCCSD, PUCCD, SUCCD

# Parameters for q-UCC antatze
num_particles = problem.num_particles
num_spin_orbitals = problem.num_spin_orbitals
n = qubit_op.num_qubits
qc = QuantumCircuit(qubit_op.num_qubits)
from qiskit.circuit import Parameter, QuantumCircuit, QuantumRegister
#the variational parameter
p=1
qubit_label = 0
# qc.ry(theta, range(n))
#qc.rz(theta, range(n))
for k in range(1):
    for i in range(n):
        theta = Parameter(f"ry_in{p}" )
        qc.ry(theta, i)
        p += 1
    for i in range(n):
        theta = Parameter(f"rz_in{p}" )
        qc.rz(theta, i)
        p += 1
#     for i in range(n):
#         theta = Parameter(f"rz_theta{p}" )
#         qc.rx(theta, i)
#         p += 1
    for i in range(n-1):
        qc.cz(i, i+1)
    for i in range(n):
        theta = Parameter(f"ry_out{p}" )
        qc.ry(theta, i)
        p += 1
    for i in range(n):
        theta = Parameter(f"rz_out{p}" )
        qc.rz(theta, i)
        p += 1
#     for i in range(n):
#         theta = Parameter(f"rz_theta{p}" )
#         qc.rx(theta, i)
#         p += 1
#qc.rz(theta, range(n))

# Add the initial state
ansatz = qc
ansatz.compose(init_state, front=True, inplace=True)

In [10]:
ansatz.draw()

┌───┐     ┌────────────┐┌────────────┐   ┌─────────────┐»
q_0: ────┤ X ├─────┤ Ry(ry_in1) ├┤ Rz(rz_in5) ├─■─┤ Ry(ry_out9) ├»
         ├───┤     ├────────────┤├────────────┤ │ └─────────────┘»
q_1: ────┤ X ├─────┤ Ry(ry_in2) ├┤ Rz(rz_in6) ├─■────────■───────»
     ┌───┴───┴────┐├────────────┤└────────────┘          │       »
q_2: ┤ Ry(ry_in3) ├┤ Rz(rz_in7) ├────────────────────────■───────»
     ├────────────┤├────────────┤                                »
q_3: ┤ Ry(ry_in4) ├┤ Rz(rz_in8) ├────────────────────────────────»
     └────────────┘└────────────┘                                »
«     ┌──────────────┐                                
«q_0: ┤ Rz(rz_out13) ├────────────────────────────────
«     ├──────────────┤┌──────────────┐                
«q_1: ┤ Ry(ry_out10) ├┤ Rz(rz_out14) ├────────────────
«     └──────────────┘├──────────────┤┌──────────────┐
«q_2: ───────■────────┤ Ry(ry_out11) ├┤ Rz(rz_out15) ├
«            │        ├──────────────┤├──────────────┤
«q_3: ───────■────────┤ Ry(ry_out12) ├┤ Rz(rz_out16) ├
«                     └──────────────┘└──────────────┘

In [13]:
from qiskit_nature.algorithms.ground_state_solvers.minimum_eigensolver_factories import NumPyMinimumEigensolverFactory
from qiskit_nature.algorithms.ground_state_solvers import GroundStateEigensolver
import numpy as np 

def exact_diagonalizer(problem, converter):
    solver = NumPyMinimumEigensolverFactory()
    calc = GroundStateEigensolver(converter, solver)
    result = calc.solve(problem)
    return result

result_exact = exact_diagonalizer(problem, converter)
exact_energy = np.real(result_exact.eigenenergies[0])
print("Exact electronic energy", exact_energy)
print(result_exact)

Exact electronic energy -1.0990640886979166
=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -8.939370137487
  - computed part:      -1.099064088698
  - FreezeCoreTransformer extracted energy part: -7.840306048789
~ Nuclear repulsion energy (Hartree): 1.05835442184
> Total ground state energy (Hartree): -7.881015715647
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  2.83458919]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  4.67568247]
    - computed part:      [0.0  0.0  4.67947432]
    - FreezeCoreTransformer extracted energy part: [0.0  0.0  -0.00379185]
  > Dipole moment (a.u.): [0.0  0.0  -1.84109328]  Total: 1.84109328
                 (debye): [0.0  0.0  -4.6795919]  Total: 4.6795919
 


In [14]:
from qiskit import Aer
backend = Aer.get_backend('statevector_simulator')
from qiskit.algorithms.optimizers import COBYLA, L_BFGS_B, SPSA, SLSQP
optimizer = COBYLA(maxiter=15000)
from qiskit.algorithms import VQE
from IPython.display import display, clear_output
def callback(eval_count, parameters, mean, std):  
    display("Evaluation: {}, Energy: {}, Std: {}".format(eval_count, mean, std))
    clear_output(wait=True)
    counts.append(eval_count)
    values.append(mean)
    params.append(parameters)
    deviation.append(std)

counts = []
values = []
params = []
deviation = []
try:
    initial_point = [0.01] * len(ansatz.ordered_parameters)
except:
    initial_point = [0.01] * ansatz.num_parameters

algorithm = VQE(ansatz,
                optimizer=optimizer,
                quantum_instance=backend,
                callback=callback,
                initial_point=initial_point)

result = algorithm.compute_minimum_eigenvalue(qubit_op)

print(result)

{   'aux_operator_eigenvalues': None,
    'cost_function_evals': 263,
    'eigenstate': array([-4.52334623e-05-3.31905235e-05j, -1.89756634e-04-3.79882335e-05j,
       -3.50253263e-05-2.55522099e-05j, -9.94301515e-01-1.06603374e-01j,
       -2.70914219e-06-1.04352700e-06j, -4.01997673e-05+3.49537398e-04j,
       -1.36395714e-09-1.06639165e-08j,  1.86857068e-05+2.23180501e-04j,
       -9.66844499e-10+1.13559465e-09j, -1.34514893e-09+5.15535299e-09j,
       -7.43468045e-10+8.79780735e-10j, -4.70791824e-06+2.61473647e-05j,
       -1.08099310e-12+6.60917771e-11j,  6.30228474e-09-5.42387526e-09j,
       -2.45995643e-11+3.11303800e-12j, -4.51640466e-07+3.35689724e-07j]),
    'eigenvalue': (-1.0814060240003027+0j),
    'optimal_circuit': None,
    'optimal_parameters': {   Parameter(rz_out14): 1.3352666291100785,
                              Parameter(rz_out13): 1.3183908530236275,
                              Parameter(ry_out9): 0.008261085257040952,
                              Parameter

In [15]:
help(ansatz)

Help on QuantumCircuit in module qiskit.circuit.quantumcircuit object:

class QuantumCircuit(builtins.object)
 |  QuantumCircuit(*regs: Union[qiskit.circuit.register.Register, int, Sequence[qiskit.circuit.bit.Bit]], name: Union[str, NoneType] = None, global_phase: Union[qiskit.circuit.parameterexpression.ParameterExpression, float] = 0, metadata: Union[Dict, NoneType] = None)
 |  
 |  Create a new circuit.
 |  
 |  A circuit is a list of instructions bound to some registers.
 |  
 |  Args:
 |      regs (list(:class:`Register`) or list(``int``) or list(list(:class:`Bit`))): The
 |          registers to be included in the circuit.
 |  
 |          * If a list of :class:`Register` objects, represents the :class:`QuantumRegister`
 |            and/or :class:`ClassicalRegister` objects to include in the circuit.
 |  
 |            For example:
 |  
 |              * ``QuantumCircuit(QuantumRegister(4))``
 |              * ``QuantumCircuit(QuantumRegister(4), ClassicalRegister(3))``
 |      

In [16]:
ansatz.draw()

┌───┐     ┌────────────┐┌────────────┐   ┌─────────────┐»
q_0: ────┤ X ├─────┤ Ry(ry_in1) ├┤ Rz(rz_in5) ├─■─┤ Ry(ry_out9) ├»
         ├───┤     ├────────────┤├────────────┤ │ └─────────────┘»
q_1: ────┤ X ├─────┤ Ry(ry_in2) ├┤ Rz(rz_in6) ├─■────────■───────»
     ┌───┴───┴────┐├────────────┤└────────────┘          │       »
q_2: ┤ Ry(ry_in3) ├┤ Rz(rz_in7) ├────────────────────────■───────»
     ├────────────┤├────────────┤                                »
q_3: ┤ Ry(ry_in4) ├┤ Rz(rz_in8) ├────────────────────────────────»
     └────────────┘└────────────┘                                »
«     ┌──────────────┐                                
«q_0: ┤ Rz(rz_out13) ├────────────────────────────────
«     ├──────────────┤┌──────────────┐                
«q_1: ┤ Ry(ry_out10) ├┤ Rz(rz_out14) ├────────────────
«     └──────────────┘├──────────────┤┌──────────────┐
«q_2: ───────■────────┤ Ry(ry_out11) ├┤ Rz(rz_out15) ├
«            │        ├──────────────┤├──────────────┤
«q_3: ───────■────────┤ Ry(ry_out12) ├┤ Rz(rz_out16) ├
«                     └──────────────┘└──────────────┘

In [17]:
parameter = result.optimal_parameters

In [43]:
parameter

{Parameter(ry_in1): 0.008239051732521298,
 Parameter(ry_in2): 0.013635804359443573,
 Parameter(ry_in3): 0.05162548888524782,
 Parameter(ry_in4): -2.1999413989625588e-05,
 Parameter(ry_out10): 0.013555948647231175,
 Parameter(ry_out11): 0.0516520269898158,
 Parameter(ry_out12): -4.555585169085397e-05,
 Parameter(ry_out9): 0.008261085257040952,
 Parameter(rz_in5): -0.010143905866716232,
 Parameter(rz_in6): -0.028142425222093263,
 Parameter(rz_in7): 0.008659821134257232,
 Parameter(rz_in8): 1.4378429162081177,
 Parameter(rz_out13): 1.3183908530236275,
 Parameter(rz_out14): 1.3352666291100785,
 Parameter(rz_out15): -0.26460520551963923,
 Parameter(rz_out16): 1.2198754282688027}

In [20]:
summ = 0
for i in qubit_op.primitive.coeffs:
    summ += abs(i*i)
print(summ)

0.5161803170096461


In [21]:
op_list = qubit_op.primitive.group_commuting()

In [23]:
def MeasureCircuit(Sum_Op):
    # Determine how many commute groups are in the SummedOp
    num_terms = len(Sum_Op)

    # Find the Paulis with least number of I in it(shoud be 0).
    # The problem is here. In this case, not not all the subgroups have a term with no I in it. So we have to loop over all
    # of the terms to construct a Pauli string.
    Pauli = ''

    for i in range(Sum_Op.num_qubits):
        intermed = []
        for j in range(num_terms):
            intermed.append(Sum_Op[j].to_label()[i])
        print(intermed)
        if 'X' in intermed:
            Pauli += 'X'
        elif 'Y' in intermed:
            Pauli += 'Y'
        else:
            Pauli += 'Z'

    if len(Pauli) != Sum_Op.num_qubits:
        raise Exception('The length does not equal, traverse has problem.')

    Pauli_string = Pauli[::-1]  # This has reversed the order.
    # Now Pauli_string is the target that we should use to construct the measurement circuit.

    qc = QuantumCircuit(Sum_Op.num_qubits, Sum_Op.num_qubits)
    qc.barrier()
    print(Pauli_string)

    for i in range(Sum_Op.num_qubits):
        if Pauli_string[i] == 'X':
            qc.u(np.pi / 2, 0, np.pi, i)
        if Pauli_string[i] == 'Y':
            qc.u(np.pi / 2, 0, np.pi / 2, i)
        else:
            None

    qc.measure(range(Sum_Op.num_qubits), range(Sum_Op.num_qubits));

    return qc


In [24]:
LiH_list = []
for commute_set in op_list:
    temp_list = []
    for i in range(0, len(commute_set.paulis)):
        temp_list.append((commute_set.paulis[i].to_label(),commute_set.coeffs[i]))
    LiH_list.append(temp_list)

In [25]:
LiH_list

[[('ZXZX', (-0.0030346568302072364+0j)),
  ('IXZX', (-0.0030346568302072364+0j)),
  ('ZXIX', (0.0030346568302072364+0j)),
  ('IXIX', (0.0030346568302072364+0j)),
  ('ZIZX', (-0.010540187409031097+0j)),
  ('ZIIX', (0.010540187409031097+0j)),
  ('ZXZI', (-0.010540187409031097+0j)),
  ('IXZI', (-0.010540187409031097+0j)),
  ('ZIZI', (-0.11409163501020526+0j)),
  ('IIZX', (-0.002734107511319098+0j)),
  ('IIIX', (0.002734107511319098+0j)),
  ('ZXII', (0.002734107511319096+0j)),
  ('IXII', (0.002734107511319096+0j)),
  ('ZIII', (-0.3713564037724+0j)),
  ('IIZI', (0.37135640377239987+0j)),
  ('IIII', (-0.19975412837028697+0j))],
 [('XXZX', (0.008373361424267884+0j)),
  ('YYZX', (-0.008373361424267884+0j)),
  ('XXIX', (-0.008373361424267884+0j)),
  ('YYIX', (0.008373361424267884+0j)),
  ('ZZZX', (0.0015406700897092698+0j)),
  ('ZZIX', (-0.0015406700897092698+0j)),
  ('XXZI', (0.03511677040240964+0j)),
  ('YYZI', (-0.03511677040240964+0j)),
  ('ZZZI', (-0.060505605672773174+0j)),
  ('XXII', (-0

In [26]:
op_list[-1].paulis

PauliList(['XZXZ', 'XIXZ', 'XZXI', 'XIXI', 'IZXZ', 'IZXI', 'XZIZ',
           'XIIZ', 'IZIZ'])

In [27]:
op_list[-1].coeffs

array([-0.00657574+0.j,  0.00657574+0.j, -0.00657574+0.j,  0.00657574+0.j,
       -0.01273391+0.j, -0.01273391+0.j, -0.01273391+0.j,  0.01273391+0.j,
       -0.12357087+0.j])

In [28]:
meas_qc = MeasureCircuit(op_list[-1].paulis)

['X', 'X', 'X', 'X', 'I', 'I', 'X', 'X', 'I']
['Z', 'I', 'Z', 'I', 'Z', 'Z', 'Z', 'I', 'Z']
['X', 'X', 'X', 'X', 'X', 'X', 'I', 'I', 'I']
['Z', 'Z', 'I', 'I', 'Z', 'I', 'Z', 'Z', 'Z']
ZXZX


In [29]:
meas_qc.draw()

░               ┌─┐         
q_0: ─░───────────────┤M├─────────
      ░ ┌────────────┐└╥┘   ┌─┐   
q_1: ─░─┤ U(π/2,0,π) ├─╫────┤M├───
      ░ └────────────┘ ║ ┌─┐└╥┘   
q_2: ─░────────────────╫─┤M├─╫────
      ░ ┌────────────┐ ║ └╥┘ ║ ┌─┐
q_3: ─░─┤ U(π/2,0,π) ├─╫──╫──╫─┤M├
      ░ └────────────┘ ║  ║  ║ └╥┘
c_0: ══════════════════╩══╬══╬══╬═
                          ║  ║  ║ 
c_1: ═════════════════════╬══╩══╬═
                          ║     ║ 
c_2: ═════════════════════╩═════╬═
                                ║ 
c_3: ═══════════════════════════╩═

In [30]:
qubit_op.num_qubits

4

In [31]:
op_list[1].paulis[0].num_qubits

4

In [32]:
op_list[1].paulis

PauliList(['XXZX', 'YYZX', 'XXIX', 'YYIX', 'ZZZX', 'ZZIX', 'XXZI',
           'YYZI', 'ZZZI', 'XXII', 'YYII', 'ZZII'])

In [33]:
op_list[0].paulis[0].to_label()

'ZXZX'

In [34]:
op_list[0].coeffs

array([-0.00303466+0.j, -0.00303466+0.j,  0.00303466+0.j,  0.00303466+0.j,
       -0.01054019+0.j,  0.01054019+0.j, -0.01054019+0.j, -0.01054019+0.j,
       -0.11409164+0.j, -0.00273411+0.j,  0.00273411+0.j,  0.00273411+0.j,
        0.00273411+0.j, -0.3713564 +0.j,  0.3713564 +0.j, -0.19975413+0.j])

In [35]:
op_list[1].coeffs

array([ 0.00837336+0.j, -0.00837336+0.j, -0.00837336+0.j,  0.00837336+0.j,
        0.00154067+0.j, -0.00154067+0.j,  0.03511677+0.j, -0.03511677+0.j,
       -0.06050561+0.j, -0.0003099 +0.j,  0.0003099 +0.j, -0.21195934+0.j])

In [38]:
paulis = qubit_op.primitive.paulis

In [40]:
coeffs = qubit_op.primitive.coeffs

In [41]:
with open(r'LiH_hamiltonian_1.5.txt', 'w') as fp:
    for i in range(len(paulis)):
        pauli_str = paulis[i].to_label()
        coeffs_str = str(coeffs[i].real)
        fp.write("{}, {}\n".format(pauli_str, coeffs_str))
    print('Done')

Done
